# The Most Common Terraform Security Issues in 1000 GitHub Open Source Projects

This Jupyter notebook is the accompanying resource to the cloudlines.de blog article [Working Title](...). Read the blog article for more details and interpretation of the results.

# Prerequesites

* install [tfsec](https://github.com/aquasecurity/tfsec) to scan the Terraform code
* you need to create a [GitHub Personal Access Token](https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/creating-a-personal-access-token) in order to access the GitHub API
* you will need the following python libraries installed
    * pandas
    * requests
    * pygit2

In [1]:
import pandas as pd
import sqlite3
import requests as r
import time
import pygit2
import glob
import os
from IPython.display import Markdown as md

In [1]:
# paste your GitHub Personal Access Token here, as described in the 'Prerequesites' section
gh_token = "<pase_your_token_here>"

In [3]:
base_url = "https://api.github.com/search/repositories?q=language:terraform&sort=stars"
attributes = ['id', 'name', 'clone_url', 'default_branch', 'stargazers_count', 'size']
repos_per_page = 100


result = r.get(base_url, params={'page': 1, 'per_page': 1})
num_repos = result.json()['total_count']
md(f"GitHub hosts **{num_repos}** Terraform repositories. Unfortunately, due to API restrictions we can only download the top **1000** of these.")

GitHub hosts **175407** Terraform repositories. Unfortunately, due to API restrictions we can only download the top **1000** of these.

# Request Data from GitHub API

First we will use the `/search/repositories` endpoint of the GitHub API to request the top 1000 Terraform repositories. The top 1000 repositories are defined by their `stargazers_count`, i.e. the number of stars a repository has. 

A SQLite database `github_terraform_repos.sql` stores the results.

In [4]:
def extract_data(items):
    data = []
    for item in items:
        date = {}
        for attr in attributes:
            date[attr] = item[attr]
        data.append(date)

    return pd.DataFrame(data)


def get_page_items(num, repos_per_page):
    result = r.get(base_url, params={
        'page': num, 
        'per_page': repos_per_page,
    }, headers={
        'Authorization': f"token {gh_token}"
    })
    if result.status_code != 200:
        print(result.status_code)
        print(result.json())
    return result.json()['items']
    

def safe_data(df, file="github_terraform_repos.sql", table="github_repos"):
    con = sqlite3.connect(file)
    df.to_sql(table, con, if_exists="append")
    con.close()

    
start_time = time.time()
total_items = 0

for i in range(1, int(1000 / repos_per_page) + 1):
    items = get_page_items(i, repos_per_page)
    df = extract_data(items)
    safe_data(df)
    total_items += len(items)
    
    print(f"{time.time() - start_time:.1f}s\ti={i} fetched={len(items)}\ttotal={total_items}")    

3.3s	i=1 fetched=100	total=100
6.5s	i=2 fetched=100	total=200
9.7s	i=3 fetched=100	total=300
12.9s	i=4 fetched=100	total=400
16.0s	i=5 fetched=100	total=500
19.7s	i=6 fetched=100	total=600
22.7s	i=7 fetched=100	total=700
25.4s	i=8 fetched=100	total=800
28.8s	i=9 fetched=100	total=900
31.7s	i=10 fetched=100	total=1000


# Clone GitHub Repositories

Next we will clone main brach (e.g. `main` or `master`) of all of the 1000 repositories to our local file storage for further analysis. The repositories will be stored the subfolder `./repositories`.

**Notes**: 
* This will take quite some time. Also, some of the repositories are quite large. This can easily take 1 - 2 hours.
* The cloning operation does not have a retry logic implemented. Due to network issues, it might be necessary to run the below cell multiple times in order to clone all repositories. 

In [5]:
start_time = time.time()
working_dir = os.getcwd()
file = "github_terraform_repos.sql"
table = "github_repos"

con = sqlite3.connect(file)
cur = con.cursor()

for i, row in enumerate(cur.execute(f"SELECT id, name, clone_url, default_branch FROM {table}")):
    _id, name, clone_url, branch = row
    print(f"{time.time() - start_time:.1f}s\ti={i} Cloning branch '{branch}' of repository '{name}'")
    
    folder = f"{working_dir}/repositories/{_id}_{name}"
    if not os.path.exists(folder):
        pygit2.clone_repository(
            clone_url,
            folder,
            checkout_branch=branch
        )  
    
con.close()

0.0s	i=0 Cloning branch 'master' of repository 'terraform-aws-eks'
0.0s	i=1 Cloning branch 'master' of repository 'terraform-aws-vpc'
0.0s	i=2 Cloning branch 'main' of repository 'ampernetacle'
0.0s	i=3 Cloning branch 'master' of repository 'stack'
0.0s	i=4 Cloning branch 'master' of repository 'terraform-up-and-running-code'
0.0s	i=5 Cloning branch 'main' of repository 'typhoon'
0.0s	i=6 Cloning branch 'master' of repository 'best-practices'
0.0s	i=7 Cloning branch 'master' of repository 'terraform-best-practices'
0.0s	i=8 Cloning branch 'master' of repository 'terraform-best-practices'
0.0s	i=9 Cloning branch 'master' of repository 'terraform-course'
0.0s	i=10 Cloning branch 'master' of repository 'okd'
0.0s	i=11 Cloning branch 'master' of repository 'terraform-guides'
0.0s	i=12 Cloning branch 'main' of repository 'boilerplates'
0.0s	i=13 Cloning branch 'main' of repository 'terraform-aws-eks-blueprints'
0.0s	i=14 Cloning branch 'master' of repository 'terraform-beginner-to-advanced-

# Run tfsec to Generate CSV Reports

Now that we downloaded all of the Terraform code to the `repositories` folder, we can run **tfsec** to find potential security issues. **tfsec** will generate a CSV report for each repository.

To make things easier we will use the following bash script. Copy the code to a file called `run_tfsec.sh`.


#### run_tfsec.sh
```bash 
#!/bin/bash

cd ./repositories

for repo in $(ls .); do
    if [ -d "$repo" ]; then
        tfsec $repo --ignore-hcl-errors --soft-fail --format csv --out "$repo.csv"
    fi
done
```

Then add permissions to execute the file and then run the script. This took ~5 minutes on my laptop.


```bash
$ chmod +x run_tfsec.sh
$ ./run_tfsec.sh
```

# Save CSV Reports in SQLite

After generating the tfsec CSV reports, we will now:
* read all CSV reports
* concatenate the reports to one big pandas dataframe
* store the result in a new SQLite database for future processing

In [6]:
working_dir = os.getcwd()
csv_files = glob.glob(f"{working_dir}/repositories/*.csv")

df_list = []
for filename in csv_files:
    temp_df = pd.read_csv(filename, index_col=None, header=0)
    df_list.append(temp_df)

df = pd.concat(df_list, axis=0, ignore_index=True)
safe_data(df, file="github_terraform_repos.sql", table="findings")

df.head()

,file,start_line,end_line,rule_id,severity,description,link,passed
0,consul.tf,60,77,aws-ec2-enable-at-rest-encryption,HIGH,Root block device is not encrypted.,https://aquasecurity.github.io/tfsec/v1.18.0/c...,False
1,consul.tf,41,58,aws-ec2-enable-at-rest-encryption,HIGH,Root block device is not encrypted.,https://aquasecurity.github.io/tfsec/v1.18.0/c...,False
2,consul.tf,60,77,aws-ec2-enforce-http-token-imds,HIGH,Instance does not require IMDS access to requi...,https://aquasecurity.github.io/tfsec/v1.18.0/c...,False
3,consul.tf,41,58,aws-ec2-enforce-http-token-imds,HIGH,Instance does not require IMDS access to requi...,https://aquasecurity.github.io/tfsec/v1.18.0/c...,False
4,main.tf,49,89,aws-ec2-enable-at-rest-encryption,HIGH,Root block device is not encrypted.,https://aquasecurity.github.io/tfsec/v1.18.0/c...,False
